# LD_DWH_CRA_HBR_MAP review

## Funkce
- get_nearest - Procedural Logic (distance calculations, Loop) **(!)**
- get_nearest2 - Static SQL (Alternative implementation - not actively used)

## Procedury
### Set parameters
- set_cfg_params Static SQL (Load configuration parameters)

### Stages
- stg_loc_upd Row-based DML + Bulk Operations **(!)**
- stg_org - Static SQL + Hierarchical queries **(! Need to be converted to Recursive CTE)**
- stg_ptorg - Static SQL
- stg_equip - Static SQL
- stg_loc - Static SQL
- stg_prod_prio - Static SQL

### HBR Algorithm Steps
- alg_step1 - Static SQL
- alg_step2 - Static SQL
- alg_step3 - Static SQL
- alg_step4 - Static SQL
- alg_step5 - Static SQL
- alg_step6 - Static SQL
- alg_step7 - Static SQL
- alg_step8 - Static SQL
- alg_step9 - Static SQL
- alg_step10 - Static SQL

### Apply Results to Target tables

- main_pt - Static SQL
- main_ptorg - Static SQL
- main_ptclsus - Static SQL

Note: There are function calls (get_nearest and set_cfg_params) used in Static SQL

In [0]:
CREATE OR REPLACE PACKAGE ETL_OWNER.LD_DWH_CRA_HBR_MAP AS

function get_nearest (pLat      number,
                      pLon      number,
                      pX        number,
                      pY        number,
                      maxDist   integer default NULL)
return integer;

/* Create stage table - list of clients for processing */
procedure stg_ptorg
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* Create stage table - list of organizations for processing */
procedure stg_org
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* Create stage table - list of card merchant equipments with location */
procedure stg_equip
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* Create stage table - locations for parties */
procedure stg_loc
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* Create stage table - find nearest org for location */
procedure stg_loc_upd
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* Create stage table - rpoduct priorities from CPT */
procedure stg_prod_prio
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* HBR ALG STEP1 - Oblibeny poradce */
procedure alg_step1
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* HBR ALG STEP2 - Rucni nastaveni VOTem */
procedure alg_step2
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* HBR ALG STEP3 - Navsteva na pobocce  */
procedure alg_step3
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* HBR ALG STEP4 - ATM, Platbomat  */
procedure alg_step4
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* HBR ALG STEP5 - Produkt zalozeny v CS */
procedure alg_step5
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* HBR ALG STEP6 - Pobocka nejbliz kontaktni adrese */
procedure alg_step6
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* HBR ALG STEP7 - Pobocka nejbliz trvale adrese */
procedure alg_step7
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* HBR ALG STEP8 - Posledni zruseny produkt */
procedure alg_step8
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* HBR ALG STEP9 - Pobocka nejbliz adrese instancnich klientu (Step6 se tyka pouze unifikovanych klientu) */
procedure alg_step9
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* HBR ALG STEP10 - Zachovani puvodni pobocky (v pripade, ze je validni) */
procedure alg_step10
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* Apply homebranch - PARTIES  */
procedure main_pt
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* Apply homebranch - PARTY_ORGANIZATIONS  */
procedure main_ptorg
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

/* Apply homebranch - PARTY_CLASSF_USAGES  */
procedure main_ptclsus
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null);

end;
/

CREATE OR REPLACE PACKAGE BODY ETL_OWNER.LD_DWH_CRA_HBR_MAP AS

 C_HB1_MONTHS               integer;
 C_HB2_MONTHS               integer;
 C_HB3_MONTHS               integer;
 C_HB_ATM_WITHDRAWAL_CNT    integer;
 C_HB_PAYM_WITHDRAWAL_CNT   integer;
 C_HB_ATM_PERIOD            integer;
 C_HB_PAYM_PERIOD           integer;
 C_HB_ATM_TIME_RANGE_FROM   varchar2(10);
 C_HB_ATM_TIME_RANGE_TO     varchar2(10);
 C_HB_PAYM_TIME_RANGE_FROM  varchar2(10);
 C_HB_PAYM_TIME_RANGE_TO    varchar2(10);
 C_HB_ATM_MAX_DISTANCE      integer;
 C_HB_PAYM_MAX_DISTANCE     integer;
 C_HB_VISIT_CNT             integer;
 C_HB_VISIT_PERIOD          integer;
 C_HB_ALG_PERIOD            integer;
 CRA_HB_RULE_5_VISIT_CNT    integer; --Pocet navstev klienta v kroku 5
 CRA_HB_RULE_5_VISIT_PERIOD integer; --Perioda, za kterou se pocita pocet navstev klienta pobocky v kroku 5. (musel mit alespon 1 navstevu za tuto periodu)
 CRA_HB_RULE_5_DISTANCE     integer; --Maximalni akceptovana vzdalenost kontaktni adresy klienta od pobocky v kroku 5. (v metrech, v parametricke tabulce je v kilometrech)

/*** PRIVATE FUNCTIONS - get nearest ***/
procedure set_cfg_params is
begin
 select max(decode(usepar_id, 'CRA_HB1_HISTORY_MONTHS', usepar_value_1, null)) as hb1,
        max(decode(usepar_id, 'CRA_HB2_HISTORY_MONTHS', usepar_value_1, null)) as hb2,
        max(decode(usepar_id, 'CRA_HB3_HISTORY_MONTHS', usepar_value_1, null)) as hb3,
        max(decode(usepar_id, 'CRA_HB_ATM_WITHDRAWAL_CNT', usepar_value_1, null)) as hb4,
        max(decode(usepar_id, 'CRA_HB_PAYM_WITHDRAWAL_CNT', usepar_value_1, null)) as hb5,
        max(decode(usepar_id, 'CRA_HB_ATM_PERIOD', usepar_value_1, null)) as hb6,
        max(decode(usepar_id, 'CRA_HB_PAYM_PERIOD', usepar_value_1, null)) as hb7,
        max(decode(usepar_id, 'CRA_HB_ATM_TIME_RANGE_FROM', usepar_value_1, null)) as hb8,
        max(decode(usepar_id, 'CRA_HB_ATM_TIME_RANGE_TO', usepar_value_1, null)) as hb9,
        max(decode(usepar_id, 'CRA_HB_PAYM_TIME_RANGE_FROM', usepar_value_1, null)) as hb10,
        max(decode(usepar_id, 'CRA_HB_PAYM_TIME_RANGE_TO', usepar_value_1, null)) as hb11,
        max(decode(usepar_id, 'CRA_HB_ATM_MAX_DISTANCE', usepar_value_1, null)) as hb12,
        max(decode(usepar_id, 'CRA_HB_PAYM_MAX_DISTANCE', usepar_value_1, null)) as hb13,
        max(decode(usepar_id, 'CRA_HB_VISIT_CNT', usepar_value_1, null)) as hb14,
        max(decode(usepar_id, 'CRA_HB_VISIT_PERIOD', usepar_value_1, null)) as hb15,
        max(decode(usepar_id, 'CRA_HB_ALG_PERIOD', usepar_value_1, null)) as hb16,
        max(decode(usepar_id, 'CRA_HB_RULE_5_VISIT_CNT', usepar_value_1, null)) as CRA_HB_RULE_5_VISIT_CNT,
        max(decode(usepar_id, 'CRA_HB_RULE_5_VISIT_PERIOD', usepar_value_1, null)) as CRA_HB_RULE_5_VISIT_PERIOD,
        max(decode(usepar_id, 'CRA_HB_RULE_5_DISTANCE', usepar_value_1*1000, null)) as CRA_HB_RULE_5_DISTANCE --prepocteno na metry
   into C_HB1_MONTHS,
        C_HB2_MONTHS,
        C_HB3_MONTHS,
        C_HB_ATM_WITHDRAWAL_CNT,
        C_HB_PAYM_WITHDRAWAL_CNT,
        C_HB_ATM_PERIOD,
        C_HB_PAYM_PERIOD,
        C_HB_ATM_TIME_RANGE_FROM,
        C_HB_ATM_TIME_RANGE_TO,
        C_HB_PAYM_TIME_RANGE_FROM,
        C_HB_PAYM_TIME_RANGE_TO,
        C_HB_ATM_MAX_DISTANCE,
        C_HB_PAYM_MAX_DISTANCE,
        C_HB_VISIT_CNT,
        C_HB_VISIT_PERIOD,
        C_HB_ALG_PERIOD,
        CRA_HB_RULE_5_VISIT_CNT,
        CRA_HB_RULE_5_VISIT_PERIOD,
        CRA_HB_RULE_5_DISTANCE
   from dwh_owner.user_config_parameters
  where usepar_group_id = 'CRA_HOMEBRANCH';

 C_HB1_MONTHS:= nvl(C_HB1_MONTHS, 12);
 C_HB2_MONTHS:= nvl(C_HB2_MONTHS, 12);
 C_HB3_MONTHS:= nvl(C_HB3_MONTHS, 12);
 C_HB_ATM_WITHDRAWAL_CNT:= nvl(C_HB_ATM_WITHDRAWAL_CNT, 5);
 C_HB_PAYM_WITHDRAWAL_CNT:= nvl(C_HB_PAYM_WITHDRAWAL_CNT, 5);
 C_HB_ATM_PERIOD:= nvl(C_HB_ATM_PERIOD, 3);
 C_HB_PAYM_PERIOD:= nvl(C_HB_PAYM_PERIOD, 3);
 C_HB_ATM_TIME_RANGE_FROM:= nvl(C_HB_ATM_TIME_RANGE_FROM, 7);
 C_HB_ATM_TIME_RANGE_TO:= nvl(C_HB_ATM_TIME_RANGE_TO, 20);
 C_HB_PAYM_TIME_RANGE_FROM:= nvl(C_HB_PAYM_TIME_RANGE_FROM, 7);
 C_HB_PAYM_TIME_RANGE_TO:= nvl(C_HB_PAYM_TIME_RANGE_TO, 20);
 C_HB_ATM_MAX_DISTANCE:= nvl(C_HB_ATM_MAX_DISTANCE, 500);
 C_HB_PAYM_MAX_DISTANCE:= nvl(C_HB_PAYM_MAX_DISTANCE, 500);
 C_HB_VISIT_CNT:= nvl(C_HB_VISIT_CNT, 3);
 C_HB_VISIT_PERIOD:= nvl(C_HB_VISIT_PERIOD, 12);
 C_HB_ALG_PERIOD:= nvl(C_HB_ALG_PERIOD, 12);
 CRA_HB_RULE_5_VISIT_CNT:= nvl(CRA_HB_RULE_5_VISIT_CNT, 1);
 CRA_HB_RULE_5_VISIT_PERIOD:= nvl(CRA_HB_RULE_5_VISIT_PERIOD, 12);
 CRA_HB_RULE_5_DISTANCE:= nvl(CRA_HB_RULE_5_DISTANCE, 30*1000);
end;

 /*** PUBLIC FUNCTIONS - get nearest ***/
/* Get nearest org - Pythagoras, use JTSK (x, y)
   WGS coordinates (lat, lon) used for optimize
*/
function get_nearest (pLat      number,
                      pLon      number,
                      pX        number,
                      pY        number,
                      maxDist   integer default NULL)
return integer
is
 c_diff_pha     number:= 0.025;      -- cca 1-2 km okruh
 c_diff_oth     number:= 0.2;        -- cca 10 km
 l_lat          number:= pLat;
 l_lon          number:= pLon;
 l_diff         number;
 l_org          integer:= -1;


begin
  if nvl(l_lat,0) = 0 or nvl(l_lon,0) = 0 then   -- nic nenajdu
     return -1;
  end if;

  if pLat between 49.9 and 50.2 and pLon between 14.1 and 14.7 then   -- jsem v Praze, mensi okruh
     l_diff:= c_diff_pha;
  else
     l_diff:= c_diff_oth;
  end if;

  loop
  
	begin
	   select org_key
		 into l_org
		 from (select distance, org_key
		         from (select /* no_query_transformation*/
							abs(sqrt((pX - src.geo_jtsk_x) * (pX - src.geo_jtsk_x) + (pY - src.geo_jtsk_y) * (pY - src.geo_jtsk_y))) as distance,
							src.org_key
					   from (select /*+ index (s, shbr_org_geo_i) */
                                    org_key, geo_jtsk_x, geo_jtsk_y
							   from etl_owner.stg_hbr_org s
							  where geo_ref_flag = 'Y'
								and geo_lat between l_lat - l_diff and l_lat + l_diff
								and geo_lon between l_lon - l_diff and l_lon + l_diff) src)
				where distance <= nvl(maxdist, distance)
			  order by distance, org_key
			  )
		where rownum = 1;

	exception
	   when no_data_found then
		  dbms_output.put_line('No data found for the given criteria.'); -- Noncompliant
	end;

    exit when l_org > 0;       -- nasel
    l_diff:= l_diff+0.2;       -- zvetsi okruh o 10 km
    exit when l_diff > 1;      -- kaslu na to, pres 50 km, at jdou k mBank
  end loop;
  return l_org;
end;

/* Get nearest org - Gonio, use WGS, not used for worse performance */
function get_nearest2 (pLat number,
                       pLon number)
return integer
is
 c_pi           number:= 3.14159265358;
 c_diff_pha     number:= 0.025;      -- cca 1-2 km okruh
 c_diff_oth     number:= 0.2;        -- cca 10 km
 l_lat          number:= pLat;
 l_lon          number:= pLon;
 l_diff         number;
 l_org          integer:= -1;
begin
  if nvl(l_lat,0) = 0 or nvl(l_lon,0) = 0 then   -- nic nenajdu
     return -1;
  end if;

  if pLat between 49.9 and 50.2 and pLon between 14.1 and 14.7 then   -- jsem v Praze, mensi okruh
     l_diff:= c_diff_pha;
  else
     l_diff:= c_diff_oth;
  end if;

  loop

    begin
       select org_key
         into l_org
         from (select /*+ no_query_transformation*/
                      6371 * acos(least((cos(c_pi * (90 - l_lat) / 180) * cos(c_pi * (90 - src.geo_lat) / 180)) +
                                        (sin(c_pi * (90 - l_lat) / 180) * sin(c_pi * (90 - src.geo_lat) / 180) * cos(c_pi * (l_lon - src.geo_lon) / 180)), 1)) as distance,
                      src.org_key
                 from (select /*+ index (s, shbr_org_geo_i) */
                              org_key, geo_lat, geo_lon
                         from etl_owner.stg_hbr_org s
                        where geo_ref_flag = 'Y'
                          and geo_lat between l_lat - l_diff and l_lat + l_diff
                          and geo_lon between l_lon - l_diff and l_lon + l_diff) src
              order by distance, org_key
              )
        where rownum = 1;

    exception
       when no_data_found then
          dbms_output.put_line('No data found for the given criteria.'); -- Noncompliant
    end;

    exit when l_org > 0;          -- nasel
    l_diff:= l_diff+0.2;          -- zvetsi okruh o 10 km
    exit when l_diff > 1;

 end loop;
 return l_org;
end;

/*** PUBLIC PROCEDURES - STAGES ***/
/* Create stage table - list of clients for processing */
procedure stg_ptorg
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is

begin
   -- truncate
 ps_global_dbms.truncate_table_pr ('STG_HBR_PARTY_ORG','ETL_OWNER');
 -- Insert data
 insert into STG_HBR_PARTY_ORG
        (pt_key,
         party_type,
         hbr_source,
         hbr_key_old,
         hbr_unified_id_old
         )
 select p.pt_key,
        pttp.pttp_unified_id as party_type,
        p.pt_organization_source_code hbr_source,
        p.org_key as old_hbr_key,
        o.org_unified_id as old_hbr_unified_id
   from dwh_owner.parties p,
        dwh_owner.party_status ptst,
        dwh_owner.party_types pttp,
        dwh_owner.organizations o
  where ptori_key = 4
    and pt_organization_source_code != 'C'
    and pt_deleted_flag = 'N'
    and p.pttp_key = pttp.pttp_key
    and p.org_key = o.org_key
    and pttp.pttp_unified_id <> 'E'   -- krome ESSO
    and p.ptstat_cs_key = ptst.ptstat_key
    --and ptst.ptstat_unified_id = 'K' -- 1,6   --zruseno v 17P DM66694
 ;

 merge
  into STG_HBR_PARTY_ORG trg
 using (
        select ptorg.PT_KEY
             , ptorg.PTORG_VALID_FROM as PROTECTION_DATE
          from dwh_owner.PARTY_ORGANIZATIONS ptorg
             , dwh_owner.PARTY_CLASSF_USAGES ptclsus
             , dwh_owner.PARTY_CLASSIFICATIONS ptcls
         where 1=1
           and ptorg.PTORI_KEY = 4
           and P_EFFECTIVE_DATE between ptorg.PTORG_VALID_FROM and ptorg.PTORG_VALID_TO
           and ptorg.PTORGTP_KEY = 1
           and ptorg.PT_KEY = ptclsus.PT_KEY
           and ptclsus.PTORI_KEY = 4
           and P_EFFECTIVE_DATE between ptclsus.PTCLSUS_VALID_FROM and ptclsus.PTCLSUS_VALID_TO
           and ptclsus.PTCLSTP_KEY = 211
           and ptclsus.PTCLS_KEY = ptcls.PTCLS_KEY
           and ptcls.PTCLSTP_KEY = 211
           and ptcls.PTCLS_SOURCE_SYSTEM_ID = 'RDS0'
           and ptcls.PTCLS_SOURCE_ID = '02'
       ) src
    on (
        trg.PT_KEY = src.PT_KEY
       )
  when matched then update
   set  trg.PROTECTION_DATE = src.PROTECTION_DATE
 ;
 commit;
                            
exception
 when others then
    rollback;
    raise;
end;

/* Create stage table - list of organizations for processing */
procedure stg_org
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is

begin
 -- truncate
 ps_global_dbms.truncate_table_pr ('STG_HBR_ORG','ETL_OWNER');

 -- Insert data - pobocky, ktere lze pouzit jako domovske
 insert into etl_owner.STG_HBR_ORG
   (org_key,
    org_source_system_id,
    org_name,
    org_unified_id,
    geo_jtsk_x,
    geo_jtsk_y,
    geo_lat,
    geo_lon,
    geo_ref_flag)
with d4_org_cra_hb as
           (
               (select distinct org_key
                  from dwh_owner.organizations
                 where org_deleted_flag = 'Y'
               connect by prior org_key = org_master_key
                 start with org_master_key in
                       (select org_key
                          from dwh_owner.organizations oh
                         where org_id2 in
                             (select usepar_value_1
                               from dwh_owner.user_config_parameters
                              where usepar_group_id = 'D4ORG'
                                and usepar_id like 'D4_ORG_ID2_%')
                       )
               minus
                select org_key
                  from dwh_owner.organizations
                 where org_deleted_flag = 'Y'
               connect by prior org_key = org_master_key
                 start with org_master_key in
                      (select org_key
                         from dwh_owner.organizations oh
                        where org_id2 in(select usepar_value_1
                                           from dwh_owner.user_config_parameters
                                          where usepar_group_id = 'D4ORG_EXCEPTIONS'
                                            and usepar_id like 'D4_ORG_ID2_%'
                                         )
                       )
                )
             union all
                select org_key
                  from dwh_owner.organizations
                 where org_unified_id in   
                     (select usepar_value_1 
                        from dwh_owner.user_config_parameters 
                       where usepar_group_id = 'CRA_HOMEBRANCH' and usepar_id like 'CRA_HB_BRANCH_EXCLUDE_%' 
                      )
            )
          select /*+ no_query_rewrite*/
                org_key,
                org_source_system_id,
                org_name,
                org_unified_id,
                nvl(loc_geo_jtsk_x,0), nvl(loc_geo_jtsk_y,0),
                nvl(loc_geo_latitude,0), nvl(loc_geo_longitude,0), 'N' as geo_ref_flag
           from dwh_owner.organizations o,
                dwh_owner.locations l
          where o.org_sales_place_flag = 'Y'                     -- je to obchodni misto
            and o.org_service_mm_flag = 'Y'                      -- mass market, C16705
            and o.org_deleted_flag = 'N'                         -- neni zrusena
            and o.org_master_hr_key > 0                          -- ma pobocku ze systemu HR
            and o.org_key not in (select org_key from d4_org_cra_hb)    -- nepatri do divize D4 a mezi CRA_HB virtualni pobocky
            and l.loc_source_system_id (+) = 'AD0'
            and o.loc_source_system_id = l.loc_source_system_id (+)
            and o.loc_key = l.loc_key (+)
          ;

 -- Oznac pobocky, co pripadaji v uvahu pro hledani podle lokace (maji souradnice, jsou z DB0 a unikatne jen prvni)
  update etl_owner.stg_hbr_org set
        geo_ref_flag  = 'Y'
  where org_key in
     (select org_key
        from (select org_key,
                     row_number() over (partition by geo_lat, geo_lon order by org_key) as rnum
                from etl_owner.stg_hbr_org
               where org_source_system_id = 'DB0'
                 and geo_lat > 0
                 and geo_lon > 0
             )
       where rnum = 1)
  ;

 commit;

exception
 when others then
    rollback;
    raise;	
end;

/* Create stage table - list of card merchant equipments with location */
procedure stg_equip
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is

begin

 set_cfg_params;

 -- truncate
 ps_global_dbms.truncate_table_pr ('STG_HBR_CARD_EQUIP','ETL_OWNER');

 --  ATM - lokace z Eiger
 insert into etl_owner.STG_HBR_CARD_EQUIP
   (cardmereq_key,
    source_id,
    atm_flag,
    paym_flag,
    equip_type,
    geo_jtsk_x,
    geo_jtsk_y,
    geo_lat,
    geo_lon)
 select eq.cardmereq_key,
        eq.cardmereq_source_id,
        eq.cardmereq_atm_flag,
        'N' as paym_flag,
        tp.cardmeretp_source_id as equip_type,
        loc_geo_jtsk_x,
        loc_geo_jtsk_y,
        loc_geo_latitude,
        loc_geo_longitude
   from dwh_owner.card_merchant_equipments eq,
        dwh_owner.card_merchant_equipment_types tp,
        dwh_owner.card_merchant_locations cl,
        dwh_owner.party_sites partition (partition_39) ps,
        dwh_owner.locations loc
  where 1=1
    --and eq.cardmereq_source_system_id = 'EG2'
    and eq.cardmereq_atm_flag = 'Y'      -- ATM
    and eq.cardmereq_deleted_flag ='N'
    and eq.cardmerc_key = 225
    and P_EFFECTIVE_DATE between eq.cardmereq_start_act_date and eq.cardmereq_end_act_date
    and eq.cardmeretp_key = tp.cardmeretp_key
    and tp.cardmeretp_desc like '%BANKOMAT%'
    and eq.cardmerlc_key = cl.cardmerlc_key
    and cl.ptori_key = ps.ptori_key     -- adresa
    and cl.ptst_key = ps.ptst_key
    and ps.loc_source_system_id = loc.loc_source_system_id      -- locations
    and ps.loc_key = loc.loc_key
    and loc.loc_geo_jtsk_x > 0   -- ma polohu
    and loc.loc_geo_jtsk_y > 0   -- ma polohu
    and eq.cardmereq_source_id not in (select usepar_default_value_1 from dwh_owner.user_config_parameters where usepar_group_id = 'CRA_HOMEBRANCH' and usepar_id like 'CRA_HB_ATM_EXCLUDE_%')
    ;

 --  Platbomat - lokace z Eiger
 insert into etl_owner.stg_hbr_card_equip
   (cardmereq_key,
    source_id,
    atm_flag,
    paym_flag,
    equip_type,
    geo_jtsk_x,
    geo_jtsk_y,
    geo_lat,
    geo_lon)
 select eq.cardmereq_key,
        eq.cardmereq_source_id,
        eq.cardmereq_atm_flag,
        'Y' as paym_flag,
        tp.cardmeretp_source_id as equip_type,
        loc_geo_jtsk_x,
        loc_geo_jtsk_y,
        loc_geo_latitude,
        loc_geo_longitude
   from dwh_owner.card_merchant_equipments eq,
        dwh_owner.card_merchant_equipment_types tp,
        dwh_owner.card_merchant_locations cl,
        dwh_owner.party_sites partition (partition_39) ps,
        dwh_owner.locations loc
  where 1=1
    --and eq.cardmereq_source_system_id = 'EG2'
    and eq.cardmereq_atm_flag = 'Y'      -- ATM
    and eq.cardmereq_deleted_flag ='N'
    and eq.cardmerc_key = 226
    and P_EFFECTIVE_DATE between eq.cardmereq_start_act_date and eq.cardmereq_end_act_date
    and eq.cardmeretp_key = tp.cardmeretp_key
    and tp.cardmeretp_key = 13
    and eq.cardmerlc_key = cl.cardmerlc_key
    and cl.ptori_key = ps.ptori_key     -- adresa
    and cl.ptst_key = ps.ptst_key
    and ps.loc_source_system_id = loc.loc_source_system_id      -- locations
    and ps.loc_key = loc.loc_key
    and loc.loc_geo_jtsk_x > 0   -- ma polohu
    and loc.loc_geo_jtsk_y > 0   -- ma polohu
    and eq.cardmereq_source_id not in (select usepar_default_value_1 from dwh_owner.user_config_parameters where usepar_group_id = 'CRA_HOMEBRANCH' and usepar_id like 'CRA_HB_ATM_EXCLUDE_%')
    ;

   -- Add missing data - ATM, lokace z KC portal
 insert into etl_owner.stg_hbr_card_equip
   (cardmereq_key,
    source_id,
    atm_flag,
    paym_flag,
    equip_type,
    geo_jtsk_x,
    geo_jtsk_y,
    geo_lat,
    geo_lon)
 select eq.cardmereq_key, eq.cardmereq_source_id, eq.cardmereq_atm_flag,
        case when eq.cardmerc_key = 226 AND eq.cardmeretp_key = 13 then 'Y' else 'N' end as paym_flag,
        tp.cardmeretp_source_id as equip_type,
        kc.cardmereq_geo_jtsk_x,
        kc.cardmereq_geo_jtsk_y,
        kc.cardmereq_geo_latitude,
        kc.cardmereq_geo_longitude
   from dwh_owner.card_merchant_equipments eq,      -- ATM from eiger
        dwh_owner.card_merchant_equipment_types tp,
        dwh_owner.card_merchant_equipments kc       -- ATM with location (KC portal)
  where eq.cardmereq_source_system_id = 'EG2'
    and eq.cardmereq_atm_flag = 'Y'      -- jen pro jistotu
    and eq.cardmereq_deleted_flag ='N'
    and P_EFFECTIVE_DATE between eq.cardmereq_start_act_date and eq.cardmereq_end_act_date
    and eq.cardmeretp_key = tp.cardmeretp_key
    and kc.cardmereq_source_system_id = 'KC0'
    and eq.cardmereq_source_id = kc.cardmereq_source_id
    and kc.cardmereq_geo_jtsk_x <> 0
    and kc.cardmereq_geo_jtsk_y <> 0
    and eq.cardmereq_key not in (select cardmereq_key
                                   from etl_owner.stg_hbr_card_equip
                                )
    and eq.cardmereq_source_id not in (select usepar_default_value_1 from dwh_owner.user_config_parameters where usepar_group_id = 'CRA_HOMEBRANCH' and usepar_id like 'CRA_HB_ATM_EXCLUDE_%')
   ;

 -- Add nearest branch (ATM)
 update etl_owner.stg_hbr_card_equip
    set nearest_org_key = LD_DWH_CRA_HBR_MAP.get_nearest (geo_lat,
                                                           geo_lon,
                                                           geo_jtsk_x,
                                                           geo_jtsk_y,
                                                           C_HB_ATM_MAX_DISTANCE)
  where nearest_org_key is null
    and atm_flag = 'Y'
    and paym_flag = 'N';
 
 -- Add nearest branch (Platbomat)
 update etl_owner.stg_hbr_card_equip
    set nearest_org_key = LD_DWH_CRA_HBR_MAP.get_nearest (geo_lat,
                                                           geo_lon,
                                                           geo_jtsk_x,
                                                           geo_jtsk_y,
                                                           C_HB_PAYM_MAX_DISTANCE)
  where nearest_org_key is null
    and atm_flag = 'Y'
    and paym_flag = 'Y';

 -- Add nearest branch unified ID
 update etl_owner.stg_hbr_card_equip
    set nearest_org_unif_id = (select org_unified_id
                                from dwh_owner.organizations
                               where org_key = nearest_org_key
                              )
  where nearest_org_unif_id is null;
  
 commit;

exception
 when others then
      rollback;
    raise;
end;


/* Create stage table - locations for parties */
procedure stg_loc
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is

begin
 -- truncate
 ps_global_dbms.truncate_table_pr ('STG_HBR_PARTY_LOC','ETL_OWNER');

  -- Insert data - kontaktni adresy
 insert into etl_owner.STG_HBR_PARTY_LOC
   (pt_key,
    loc_key,
    loc_type,
    geo_jtsk_x,
    geo_jtsk_y,
    geo_lat,
    geo_lon )
 select pt.pt_key,
        adr.loc_key,
        'K' as loc_type,
        adr.loc_geo_jtsk_x,
        adr.loc_geo_jtsk_y,
        adr.loc_geo_latitude,
        adr.loc_geo_longitude
   from(select pt_key
          from etl_owner.stg_hbr_party_org
         where party_type <> 'E'
        ) pt,
        (select pt_key,
                loc_key,
                loc_geo_jtsk_x,
                loc_geo_jtsk_y,
                loc_geo_latitude,
                loc_geo_longitude
                ,row_number () over (partition by pt_key order by valid_from desc) as rnum
           from (select /*+full(inst) full(ps) full(loc)*/ distinct
                        inst.pt_unified_key as pt_key,
                        loc.loc_key,
                        ptst_valid_from as valid_from,
                        loc_geo_jtsk_x,
                        loc_geo_jtsk_y,
                        loc_geo_latitude,
                        loc_geo_longitude
                   from dwh_owner.parties inst,
                        dwh_owner.party_sites ps,
                        dwh_owner.party_site_types pst,
                        dwh_owner.locations loc
                  where 1=1
                    and inst.ptori_key = 4
                    and inst.pt_key = ps.pt_key
                    and inst.ptori_key = ps.ptori_key
                    and inst.pt_deleted_flag = 'N'
                    and p_effective_date between ps.ptst_valid_from and ps.ptst_valid_to
                    and ps.ptstp_key = pst.ptstp_key
                    and pst.ptstp_unified_id = 'K'
                    and ps.loc_source_system_id = loc.loc_source_system_id
                    and ps.loc_key = loc.loc_key
                    and loc.loc_source_system_id = 'AD0'
                    and loc.loc_geo_jtsk_x > 0
                    and loc.loc_geo_jtsk_y > 0
               ) x
         ) adr
   where pt.pt_key = adr.pt_key
     and adr.rnum = 1
   ;

 -- Insert data - trvale adresy
 insert into etl_owner.stg_hbr_party_loc
        (pt_key,
         loc_key,
         loc_type,
         geo_jtsk_x,
         geo_jtsk_y,
         geo_lat,
         geo_lon
        )
         select /*+ dynamic_sampling(5) */
                pt.pt_key, adr.loc_key,
                'T' as loc_type,
                adr.loc_geo_jtsk_x,
                adr.loc_geo_jtsk_y,
                adr.loc_geo_latitude,
                adr.loc_geo_longitude
           from(select pt_key
                 from etl_owner.stg_hbr_party_org         -- jen tam, kde jeste nemam adresu
                where party_type <> 'E'
                minus
               select pt_key
                 from etl_owner.stg_hbr_party_loc
                ) pt,
                (select pt_key,
                        loc_key,
                        loc_geo_jtsk_x,
                        loc_geo_jtsk_y,
                        loc_geo_latitude,
                        loc_geo_longitude
                        ,row_number () over (partition by pt_key order by valid_from desc) as rnum
                   from(select /*+full(inst) full(ps) full(loc)*/ inst.pt_unified_key as pt_key,
                               loc.loc_key,
                               ptst_valid_from as valid_from,
                               loc_geo_jtsk_x,
                               loc_geo_jtsk_y,
                               loc_geo_latitude,
                               loc_geo_longitude
                           from dwh_owner.parties inst,
                                dwh_owner.party_sites ps,
                                dwh_owner.party_site_types pst,
                                dwh_owner.locations loc
                          where inst.ptori_key = 4
                            and inst.pt_key = ps.pt_key
                            and inst.ptori_key = ps.ptori_key
                            and inst.pt_deleted_flag = 'N'
                            and p_effective_date between ps.ptst_valid_from
                            and ps.ptst_valid_to
                            and ps.ptstp_key = pst.ptstp_key
                            and pst.ptstp_unified_id = 'T'
                            and ps.loc_source_system_id = loc.loc_source_system_id
                            and ps.loc_key = loc.loc_key
                            and loc.loc_source_system_id = 'AD0'
                            and loc.loc_geo_jtsk_x > 0
                            and loc.loc_geo_jtsk_y > 0
                       ) x
                 ) adr
           where pt.pt_key = adr.pt_key
             and adr.rnum = 1
         ;

 commit;

exception
 when others then
    rollback;
    raise;
end;


/* Create stage table - find nearest org for location */
procedure stg_loc_upd
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is
  type t_num is table of integer index by binary_integer;
  l_loc                  t_num;
  l_org                  t_num;
  i                      integer:= 0;
  c_lim      constant    integer:= 10000;

  procedure upd_org is
    bulk_errors exception;
    pragma exception_init(bulk_errors, -24381);
  begin
     forall j in 1 .. l_loc.count
        update etl_owner.stg_hbr_party_loc
          set  nearest_org_key = l_org(j)
        where loc_key = l_loc(j);
     l_loc.delete;
     l_org.delete;
     l_cnt:= l_cnt + i;
     i:= 0;
  exception
    when bulk_errors then
      rollback;
  end;
begin

 for rc in
    (select distinct
            loc_key,
            geo_jtsk_x,
            geo_jtsk_y,
            geo_lat,
            geo_lon
       from etl_owner.stg_hbr_party_loc
      where nearest_org_key is null          -- restart possible
    )
 loop
     i:= i+1;
     l_loc(i):= rc.loc_key;
     l_org(i):= get_nearest (rc.geo_lat, rc.geo_lon, rc.geo_jtsk_x, rc.geo_jtsk_y);
     if i >= c_lim then
        upd_org;
     end if;
 end loop;

 if i >= 0 then
    upd_org;
 end if;

 commit;

exception
 when others then
    rollback;
    raise;
end;


/* Create stage table - rpoduct priorities from CPT */
procedure stg_prod_prio
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is

begin

 -- truncate
 ps_global_dbms.truncate_table_pr ('STG_HBR_PROD_PRIORITY','ETL_OWNER');


insert
  into STG_HBR_PROD_PRIORITY (
       PROD_KEY,
       PRODUCT_ID,
       PROD_PRIORITY_GROUP
)
SELECT PROD_KEY
     , PRODUCT_ID
     , PROD_PRIORITY_GROUP
  FROM (
        SELECT prod.PROD_KEY
             , prod.PRODH_SOURCE_SYSTEM_ID||'.'||prod.PRODH_SOURCE_ID as PRODUCT_ID
             , replace(fp_par.PRODUCT_ID,'PRIO_GRP_','') as PROD_PRIORITY_GROUP
             , row_number() over (partition by prod.PRODH_SOURCE_SYSTEM_ID||'.'||prod.PRODH_SOURCE_ID, replace(fp_par.PRODUCT_ID,'PRIO_GRP_','') order by pr.PRODR_DELETED_FLAG) as RN
          FROM product_owner.PRODUCT_REL_TYPE prt
             , product_owner.PRODUCT_RELATIONSHIP pr
             , product_owner.FINANCIAL_PRODUCT fp_child
             , product_owner.FINANCIAL_PRODUCT fp_par
             , dwh_owner.PRODUCT_HISTORY prod
         WHERE 1=1
           AND upper(prt.ID_RELATIONSHIP_TYPE) = 'PRIO_GRP'
           AND P_EFFECTIVE_DATE BETWEEN prt.VALID_FROM AND prt.VALID_TO
           AND pr.RELATIONSHIP_TYPE = prt.ID_RELATIONSHIP_TYPE
           AND P_EFFECTIVE_DATE BETWEEN pr.VALID_FROM AND pr.VALID_TO
           AND fp_child.ID_FINANCIAL_PRODUCT = pr.fid_child
           AND P_EFFECTIVE_DATE BETWEEN fp_child.VALID_FROM AND fp_child.VALID_TO
           AND fp_par.ID_FINANCIAL_PRODUCT = pr.fid_parent
           AND P_EFFECTIVE_DATE BETWEEN fp_par.VALID_FROM AND fp_par.VALID_TO
           AND fp_child.PRODUCT_ID = prod.PRODH_SOURCE_SYSTEM_ID || '_' || prod.PRODH_SOURCE_ID
           AND P_EFFECTIVE_DATE between prod.PRODH_VALID_FROM and prod.PRODH_VALID_TO
       )
 WHERE RN = 1;

 commit;

exception
 when others then
    rollback;
    raise;
end;

/*** PUBLIC PROCEDURES - FIND HOMEBRANCH ***/
/* HBR ALG STEP1 - Oblibeny poradce */
procedure alg_step1
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is
  l_RESPTP_KEY   number;
begin

 set_cfg_params;

 select RESPTP_KEY 
  into l_RESPTP_KEY
  from DWH_OWNER.RESPONSIBILITY_TYPES 
  where RESPTP_SOURCE_SYSTEM_ID = 'CR0' 
    and RESPTP_SOURCE_ID = 'AD/MB';  

 merge --+full(trg)
  into etl_owner.STG_HBR_PARTY_ORG  trg
 using (
        select PT_KEY
             , ORG_KEY
             , ORG_UNIFIED_ID
          from (
                select pt.PT_KEY
                     , o.ORG_KEY
                     , o.ORG_UNIFIED_ID
                     , row_number() over (partition by pt.PT_KEY order by respasg.RESPASG_VALID_FROM desc) as RN
                  from (
                        select PT_KEY, PARTY_TYPE
                          from etl_owner.STG_HBR_PARTY_ORG
                         where HBR_KEY_NEW is null
                           and PARTY_TYPE in ('F','P')
                       ) pt
                     , dwh_owner.RESPONSIBILITY_ASSIGNMENTS respasg
                     , dwh_owner.RESPONSIBILITY_HISTORY resp
                     , etl_owner.STG_HBR_ORG o
                 where 1=1
                   and pt.PT_KEY = respasg.PT_KEY
                   and respasg.PTORI_KEY = 4
                   --DA-28129
                   and respasg.RESPCT_KEY = 12   --AD
                   and (    pt.PARTY_TYPE = 'P'
                         or respasg.RESPTP_KEY = l_RESPTP_KEY 
                       )  
                   --end DA-28129
                   and P_EFFECTIVE_DATE between respasg.RESPASG_VALID_FROM and respasg.RESPASG_VALID_TO
                   and respasg.RESP_KEY = resp.RESP_KEY
                   and P_EFFECTIVE_DATE between resp.RESPH_VALID_FROM and resp.RESPH_VALID_TO
                   and resp.ORG_KEY = o.ORG_KEY   -- jen ty, co mohou byt HBR
               )
         where RN = 1
       ) src
    on (
        trg.pt_key = src.pt_key
       )
  when matched then
update
   set  trg.HBR_KEY_NEW = src.ORG_KEY,
        trg.HBR_UNIFIED_ID_NEW = src.ORG_UNIFIED_ID,
        trg.ALG_STEP = '01'
 ;

 commit;

exception
 when others then
    rollback;
    raise;
end;

/* HBR ALG STEP2 - Rucni nastaveni VOTem */
procedure alg_step2
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is

begin

 set_cfg_params;

 merge --+full(trg)
  into etl_owner.STG_HBR_PARTY_ORG  trg
 using (
        select pt.PT_KEY
             , o.ORG_KEY
             , o.ORG_UNIFIED_ID
          from (
                select PT_KEY
                  from etl_owner.STG_HBR_PARTY_ORG
                 where HBR_KEY_NEW is null
               ) pt
             , dwh_owner.PARTY_ORGANIZATIONS ptorg
             , etl_owner.STG_HBR_ORG o
         where 1=1
           and pt.PT_KEY = ptorg.PT_KEY
           and ptorg.PTORI_KEY = 4
           and ptorg.PTORG_SOURCE_SYSTEM_ID = 'CR0'
           and ptorg.PTORGTP_KEY = 4
           and P_EFFECTIVE_DATE between ptorg.PTORG_VALID_FROM and ptorg.PTORG_VALID_TO
           and ptorg.ORG_KEY = o.ORG_KEY   -- jen ty, co mohou byt HBR
       ) src
    on (
        trg.pt_key = src.pt_key
       )
  when matched then
update
   set  trg.HBR_KEY_NEW = src.ORG_KEY,
        trg.HBR_UNIFIED_ID_NEW = src.ORG_UNIFIED_ID,
        trg.ALG_STEP = '02'
 ;

 commit;

exception
 when others then
    rollback;
    raise;
end;

/* HBR ALG STEP3 - Navsteva na pobocce  */
procedure alg_step3
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is

begin

  set_cfg_params;

 merge --+full(trg)
  into etl_owner.STG_HBR_PARTY_ORG  trg
 using (
        select PT_KEY
             , ORG_KEY
             , ORG_UNIFIED_ID
          from (
                select PT_KEY
                     , ORG_KEY
                     , ORG_UNIFIED_ID
                     , CNT
                     , row_number() over (partition by PT_KEY order by CNT desc, ORG_KEY asc) as RN
                  from (
                        select distinct
                               pt.PT_KEY
                             , act.ORG_KEY
                             , o.ORG_UNIFIED_ID
                             , count(1) over (partition by pt.PT_KEY, act.ORG_KEY order by 1) as cnt
                          from (select PT_KEY
                                  from etl_owner.STG_HBR_PARTY_ORG
                                 where HBR_KEY_NEW is null
                                   and add_months(P_EFFECTIVE_DATE, -1*C_HB_ALG_PERIOD) > nvl(PROTECTION_DATE, date '1000-01-01')
                                   and PARTY_TYPE <> 'E'
                               ) pt
                             , dwh_owner.ACTIVITIES act
                             , etl_owner.STG_HBR_ORG o
                         where 1=1
                           and pt.PT_KEY = act.PT_CLIENT_KEY
                           and act.PTORI_CLIENT_KEY = 4
                           and act.ACTIVST_KEY = 281
                           and act.ACT_CREATED_DATE between add_months(P_EFFECTIVE_DATE, -1*C_HB_VISIT_PERIOD) and P_EFFECTIVE_DATE
                           and act.ORG_KEY = o.ORG_KEY    -- jen ty, co mohou byt HBR
                       )
                 where CNT >= C_HB_VISIT_CNT
               )
         where RN = 1
       ) src
    on (
        trg.PT_KEY = src.PT_KEY
       )
  when matched then
update
   set  trg.HBR_KEY_NEW = src.ORG_KEY,
        trg.HBR_UNIFIED_ID_NEW = src.ORG_UNIFIED_ID,
        trg.ALG_STEP = '03'
 ;

 commit;

exception
 when others then
    rollback;
    raise;
end;

/* HBR ALG STEP4 - ATM, Platbomat  */
procedure alg_step4
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is

begin

 set_cfg_params;

 merge --+ full(tgt)
  into etl_owner.STG_HBR_PARTY_ORG  tgt
 using (
        select pt_key, org_key, org_unified_id
          from (
                select x.pt_key, x.nearest_org_key as org_key, x.nearest_org_unif_id as org_unified_id,
                       row_number() over (partition by x.pt_key order by x.cnt_trans desc, x.last_transact_date desc ) as rnum
                  from (
                        select --+ leading (pt) full(inst) full(c) full(trn) full(eq)
                               pt.pt_key, trn.cardmereq_key, trn.nearest_org_key, trn.nearest_org_unif_id, trn.trntype,
                               count(*) as cnt_trans, max(trn.cardtr_trans_local_datetime) as last_transact_date
                          from (
                                select pt_key
                                  from etl_owner.STG_HBR_PARTY_ORG
                                 where hbr_key_new is null
                                   and add_months(P_EFFECTIVE_DATE, -1*C_HB_ALG_PERIOD) > nvl(PROTECTION_DATE, date '1000-01-01')
                                   and party_type = 'F'
                               ) pt    -- pouze fyz osoby
                             , dwh_owner.PARTIES inst
                             , dwh_owner.CARDS c
                             , (
                                select trn.card_key, trn.cardmereq_key, eq.nearest_org_key, eq.nearest_org_unif_id, trn.cardtr_trans_local_datetime, 'ATM' as trntype
                                  from dwh_owner.card_transactions trn
                                     , etl_owner.STG_HBR_CARD_EQUIP eq
                                 where trn.cardtr_process_date between add_months(P_EFFECTIVE_DATE, -1*C_HB_ATM_PERIOD) and P_EFFECTIVE_DATE
                                   and to_date(to_char(trn.cardtr_trans_local_datetime,'hh24')||to_char(trn.cardtr_trans_local_datetime,'mi'),'hh24mi') between to_date(C_HB_ATM_TIME_RANGE_FROM,'hh24mi') and to_date(C_HB_ATM_TIME_RANGE_TO,'hh24mi')
                                   and trn.cardmereq_key = eq.cardmereq_key
                                   and trn.cardtr_merchant_country in ( 'CZE', 'CZ')
                                   and eq.atm_flag = 'Y'
                                   and eq.paym_flag = 'N'
                                   and eq.nearest_org_key <> -1 -- 21.05.2018 MP
                                union all
                                select trn.card_key, trn.cardmereq_key, eq.nearest_org_key, eq.nearest_org_unif_id, trn.cardtr_trans_local_datetime, 'PAYM' as trntype
                                  from dwh_owner.card_transactions trn
                                     , etl_owner.STG_HBR_CARD_EQUIP eq--
                                 where trn.cardtr_process_date between add_months(P_EFFECTIVE_DATE, -1*C_HB_PAYM_PERIOD) and P_EFFECTIVE_DATE
                                   and to_date(to_char(trn.cardtr_trans_local_datetime,'hh24')||to_char(trn.cardtr_trans_local_datetime,'mi'),'hh24mi') between to_date(C_HB_PAYM_TIME_RANGE_FROM,'hh24mi') and to_date(C_HB_PAYM_TIME_RANGE_TO,'hh24mi')
                                   and trn.cardmereq_key = eq.cardmereq_key
                                   and trn.cardtr_merchant_country in ( 'CZE', 'CZ')
                                   and eq.atm_flag = 'Y'
                                   and eq.paym_flag = 'Y'
                                   and eq.nearest_org_key <> -1 -- 21.05.2018 MP
                               ) trn
                         where inst.ptori_key in (5)
                           and inst.pt_unified_key = pt.pt_key and inst.ptori_unified_key = 4
                           and inst.pt_key = c.pt_key and inst.ptori_key = c.ptori_key
                           and c.card_source_system_id in ('EG0')
                           and c.card_key = trn.card_key
                        group by pt.pt_key, trn.cardmereq_key, trn.nearest_org_key, trn.nearest_org_unif_id, trn.trntype
                       ) x
                 where x.CNT_TRANS >= case when x.TRNTYPE = 'ATM' then C_HB_ATM_WITHDRAWAL_CNT else C_HB_PAYM_WITHDRAWAL_CNT end
               )
         where rnum = 1
       ) src
    on (tgt.pt_key = src.pt_key)
  when matched then update
   set hbr_key_new = src.org_key,
       hbr_unified_id_new = src.org_unified_id,
       alg_step = '04'
 ;

 commit;

exception
 when others then
    rollback;
    raise;
end;

/* HBR ALG STEP5 - Produkt zalozeny v CS */
procedure alg_step5
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is

begin

 set_cfg_params;

 merge /*+ full(tgt) cardinality (src, 1000000) */
  into etl_owner.STG_HBR_PARTY_ORG  tgt
 using
 (select
        pobocka.pt_key,
        pobocka.org_key,
        pobocka.org_unified_id,
        --slo by naprasit do 1 outer joinu, ale radeji prehledneji:
        CASE
            WHEN visits.CNT >= CRA_HB_RULE_5_VISIT_CNT THEN 'Y' --pokud je pocet navstev >= CRA_HB_RULE_5_VISIT_CNT, prirad pobocku. Pokud NULL tak nenavstivil pobocku, zkus dalsi krok
            WHEN contacts.PT_KEY is null and perman.PT_KEY is null THEN 'Y' -- pokud nevyplnena kontaktni ani trvala adresa, chceme ponechat produktovy vyber podle kroku 5
            WHEN contacts.PT_KEY is not null THEN -- pokud je vyplnena kontaktni adresa
                CASE 
                 --WHEN contacts.loc_geo_jtsk_x > 0 and contacts.loc_geo_jtsk_y > 0 -- nemelo by nastat z definice vypoctu STG_HBR_PARTY_LOC
                 --optimalizace, at nepocita odmocninu pro vsechny radky
                    WHEN pobocka.org_GEO_JTSK_X between contacts.GEO_JTSK_X - CRA_HB_RULE_5_DISTANCE and contacts.GEO_JTSK_X + CRA_HB_RULE_5_DISTANCE
                        AND pobocka.org_GEO_JTSK_Y between contacts.GEO_JTSK_Y - CRA_HB_RULE_5_DISTANCE and contacts.GEO_JTSK_Y + CRA_HB_RULE_5_DISTANCE
                        AND contacts.GEO_JTSK_X between pobocka.org_GEO_JTSK_X - CRA_HB_RULE_5_DISTANCE and pobocka.org_GEO_JTSK_X + CRA_HB_RULE_5_DISTANCE
                        AND contacts.GEO_JTSK_Y between pobocka.org_GEO_JTSK_Y - CRA_HB_RULE_5_DISTANCE and pobocka.org_GEO_JTSK_Y + CRA_HB_RULE_5_DISTANCE
                        -- samotna vzdalenost pytagorovou vetou
                        AND sqrt ( (pobocka.org_GEO_JTSK_X -contacts.GEO_JTSK_X) * (pobocka.org_GEO_JTSK_X -contacts.GEO_JTSK_X)  +
                           (pobocka.org_GEO_JTSK_Y -contacts.GEO_JTSK_Y) * (pobocka.org_GEO_JTSK_Y -contacts.GEO_JTSK_Y) ) <= CRA_HB_RULE_5_DISTANCE
                    THEN 'Y'
                    ELSE 'N' --  pokud je kontaktni adresa dohledana a > 30km
                END
            WHEN perman.PT_KEY is not null THEN -- pokud je vyplnena trvala adresa
                 --...overeni te same podminky pro trvalou adresu (vzdalenost <= CRA_HB_RULE_5_DISTANCE) 
                CASE WHEN pobocka.org_GEO_JTSK_X between perman.GEO_JTSK_X - CRA_HB_RULE_5_DISTANCE and perman.GEO_JTSK_X + CRA_HB_RULE_5_DISTANCE
                    AND pobocka.org_GEO_JTSK_Y between perman.GEO_JTSK_Y - CRA_HB_RULE_5_DISTANCE and perman.GEO_JTSK_Y + CRA_HB_RULE_5_DISTANCE
                    AND perman.GEO_JTSK_X between pobocka.org_GEO_JTSK_X - CRA_HB_RULE_5_DISTANCE and pobocka.org_GEO_JTSK_X + CRA_HB_RULE_5_DISTANCE
                    AND perman.GEO_JTSK_Y between pobocka.org_GEO_JTSK_Y - CRA_HB_RULE_5_DISTANCE and pobocka.org_GEO_JTSK_Y + CRA_HB_RULE_5_DISTANCE
                    -- samotna vzdalenost pytagorovou vetou
                    AND sqrt ( (pobocka.org_GEO_JTSK_X -perman.GEO_JTSK_X) * (pobocka.org_GEO_JTSK_X -perman.GEO_JTSK_X)  +
                       (pobocka.org_GEO_JTSK_Y -perman.GEO_JTSK_Y) * (pobocka.org_GEO_JTSK_Y -perman.GEO_JTSK_Y) ) <= CRA_HB_RULE_5_DISTANCE
                THEN 'Y'
                ELSE 'N' --  pokud je kontaktni adresa nedohledana, trvala adresa dohledana a > 30km
            END
            --obe adresy jsou null:
            ELSE 'Y' -- sem by se to nemelo dostat, podminka je jiz vyse
        END update_flag
  from (
    select /*+ no_query_rewrite*/
        pt_key,
        org_key,
        org_unified_id,
        org_GEO_JTSK_X,
        org_GEO_JTSK_Y
     from (select pt_key,
                org_key,
                org_unified_id,
                org_GEO_JTSK_X,
                org_GEO_JTSK_Y,
                row_number() over (partition by pt_key order by pri1, pri2, pri3 desc, pri4, pri5 desc, pri6 desc, pri7 desc, org_key ) as rnum
          from  (select s.pt_key,
                        s.org_key,
                        s.org_unified_id,
                        s.pri1,
                        s.pri2,
                        s.pri3,
                        s.pri4,
                        s.org_GEO_JTSK_X,
                        s.org_GEO_JTSK_Y,
                        count(*)       over (partition by s.pt_key, s.org_key, s.pri1, s.pri2) as pri5,  -- 6. pocet uctu na pobocce se stejnym vztahem a prioritou produktu
                        sum(s.acc_bal) over (partition by s.pt_key, s.org_key, s.pri1, s.pri2) as pri6,  -- 7. suma zustatku na pobocce se stejnym vztahem a prioritou produktu
                        max(s.acc_bal) over (partition by s.pt_key, s.org_key, s.pri1, s.pri2) as pri7   -- 8. max zustatek na pobocce se stejnym vztahem a prioritou produktu
                  from  -- source select
                        (select /*+ cardinality (pt, 3000000) */
                                pt.pt_key,
                                acc.acctp_key,
                                acc.acc_key,
                                acc.org_key as org_key,
                                acc.org_unified_id,
                                nvl(bal.amt,0) as acc_bal,
                                acc.org_GEO_JTSK_X,
                                acc.org_GEO_JTSK_Y,
                                --rel.rel_pri as pri1,                                            -- 1.poradi zdroje --zruseno v 17P
                                case when rel.rel_akt = 'Y' and acc.close_flag = 'N' and rel.rel_typ = '42' then 1       -- aktivni ucet 42
                                     when rel.rel_akt = 'Y' and acc.close_flag = 'N' and rel.rel_typ = '45' then 2       -- aktivni ucet 45
                                     when rel.rel_akt = 'Y' and acc.close_flag = 'N' then 3       -- aktivni ucet ostatni
                                     else 4
                                end as pri1,                                                     -- 1. vztah k uctu
                                acc.prod_pri as pri2,                                            -- 2. priorita produktu
                                rel.rel_from as pri3,                                            -- 3. platnost vztahu
                                case when acc.kontokorent = 'Y' then 1 else 2 end as pri4        -- 4. kontokorent
                           from (select pt_key
                                   from etl_owner.STG_HBR_PARTY_ORG
                                  where hbr_key_new is null
                                    and add_months(P_EFFECTIVE_DATE, -1*C_HB_ALG_PERIOD) > nvl(PROTECTION_DATE, date '1000-01-01')
                                    and party_type <> 'E'           -- jeste nemaji pobocku
                                 ) pt,
                                 (select pt_key,
                                         ptori_key,
                                         pt_unified_key,
                                         ptori_unified_key
                                    from dwh_owner.parties
                                   where ptori_key not in (4,99)         -- historicke instance ne
                                     and pt_deleted_flag = 'N'
                                 ) inst,
                                 (select pt_key,
                                         ptori_key,
                                         acc_key,
                                         acctp_key,
                                         accpt_valid_from as rel_from,
                                         case when p_effective_date between accpt_valid_from and accpt_valid_to
                                                   then 'Y'
                                              else 'N'
                                         end as rel_akt,
                                         accpttp_unified_id as rel_typ,
                                         case when ss.srcsys_priority_group < 0
                                                   then 999
                                              else ss.srcsys_priority_group
                                         end as rel_pri
                                    from dwh_owner.account_parties ap,
                                         dwh_owner.account_party_types apt,
                                         dwh_owner.source_systems ss
                                   where ap.accpttp_key = apt.accpttp_key
                                     and ap.accpt_source_system_id = ss.srcsys_id
                                  ) rel,
                                 (select /*+ full(o)*/
                                         a.acc_key,
                                         a.acctp_key,
                                         a.org_key,
                                         o.org_unified_id,
                                         a.acc_close_flag as close_flag,
                                         case when nvl(p.prod_priority_group, 999) = 0 then 999 else nvl(p.prod_priority_group, 999) end as prod_pri,
                                         case when det.acctp_key in (101,202,602) and det.acc_overdraft_flag = 'Y'
                                                   then 'Y'
                                              else 'N'
                                         end as kontokorent,
                                         o.GEO_JTSK_X as org_GEO_JTSK_X,
                                         o.GEO_JTSK_Y as org_GEO_JTSK_Y
                                    from dwh_owner.all_accounts a,
                                         (select *
                                           from dwh_owner.accounts
                                          where acctp_key in (101,202,602) ) det,
                                         (select prod_key, prod_priority_group
                                            from etl_owner.STG_HBR_PROD_PRIORITY ) p,
                                          etl_owner.stg_hbr_org o
                                    where a.org_key > 0
                                      and a.org_key = o.org_key       -- jen ty, co mohou byt domovska pobocka
                                      and a.acctp_key not in (9999)   -- historicke ucty ne
                                      and a.prod_key = p.prod_key (+)
                                      and a.acc_deleted_flag = 'N'
                                      and a.acctp_key = det.acctp_key (+)
                                      and a.acc_key = det.acc_key (+)
                                    ) acc,
                                   (select /*+ full (acb) */
                                           acc_key,
                                           acctp_key,
                                           abs(acurbal_amount_czk) as amt
                                      from dwh_owner.account_current_balances acb
                                     where accbaltp_key = 33  -- current balance
                                    ) bal
                              where pt.pt_key = inst.pt_unified_key
                                and 4 = inst.ptori_unified_key
                                and inst.pt_key = rel.pt_key
                                and inst.ptori_key = rel.ptori_key
                                and rel.acc_key = acc.acc_key
                                and rel.acctp_key = acc.acctp_key
                                and acc.acc_key = bal.acc_key (+)
                                and acc.acctp_key = bal.acctp_key (+)
                         ) s
       )  -- posledni 3 priority
     )  -- rownum
   where rnum = 1  -- get first
   ) pobocka,
   --pro vybranou pobocku overit, ze ji klient navstivil za posledni rok a jeji vzdalenost je do 30KM
   -- schvalne az na konci, podle pozadavku. Navic overeni vzdalenosti by mohlo bezet dlouho
        -- navstivil klient pobocku za posl. X (12 mesicu)
        -- navstevy z kroku alg_step3: (navstevy muzou byt na jinou ORG_KEY, my chceme na ORG_UNIFIED_ID)
    (
            select act.PT_CLIENT_KEY as PT_KEY
                 --, act.ORG_KEY
                 , o.ORG_UNIFIED_ID
                 , count(1) cnt --over (partition by act.PT_CLIENT_KEY, act.ORG_KEY order by 1) as cnt
              from dwh_owner.ACTIVITIES act
                 , etl_owner.STG_HBR_ORG o
             where 1=1
               --and pt.PT_KEY = act.PT_CLIENT_KEY
               and act.PTORI_CLIENT_KEY = 4
               and act.ACTIVST_KEY = 281
               and act.ACT_CREATED_DATE between add_months(P_EFFECTIVE_DATE, -1*CRA_HB_RULE_5_VISIT_PERIOD) and P_EFFECTIVE_DATE
               and act.ORG_KEY = o.ORG_KEY    -- jen ty, co mohou byt HBR
             GROUP BY act.PT_CLIENT_KEY --as PT_KEY
                 --, act.ORG_KEY
                 , o.ORG_UNIFIED_ID
             --having count(*) >= CRA_HB_RULE_5_VISIT_CNT --alespon jedna navsteva
        ) visits,
        (SELECT * FROM etl_owner.STG_HBR_PARTY_LOC contacts WHERE loc_type='K') contacts,
        (SELECT * FROM etl_owner.STG_HBR_PARTY_LOC contacts WHERE loc_type='T') perman
        WHERE 1=1
          --pripojeni navstev PT_KEY na pobocce ORG_UNIFIED_ID
          AND pobocka.org_unified_id = visits.org_unified_id (+)
          AND pobocka.pt_key = visits.pt_key (+)
          -- pripojeni kontaktnich adres s geo lokaci:
          AND pobocka.PT_KEY = contacts.PT_KEY (+)
          -- pripojeni trvalych adres s geo lokaci:
          AND pobocka.PT_KEY = perman.PT_KEY (+)
 ) src
 on
  (tgt.pt_key = src.pt_key
    and src.update_flag='Y')
 when matched then update set
     hbr_key_new = src.org_key,
     hbr_unified_id_new = src.org_unified_id,
     alg_step = '05'
 where src.update_flag = 'Y'
 ;

 commit;

exception
 when others then
    rollback;
    raise;
end;

/* HBR ALG STEP6 - Pobocka nejbliz kontaktni adrese */
procedure alg_step6
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is

begin

 set_cfg_params;

 merge --+ full(tgt)
  into etl_owner.STG_HBR_PARTY_ORG  tgt
 using (
        select pt.pt_key, loc.nearest_org_key as org_key, org.org_unified_id
          from (
                select pt_key from etl_owner.STG_HBR_PARTY_ORG
                 where hbr_key_new is null
                   and add_months(P_EFFECTIVE_DATE, -1*C_HB_ALG_PERIOD) > nvl(PROTECTION_DATE, date '1000-01-01')
                   and party_type <> 'E'
               ) pt
             , etl_owner.STG_HBR_PARTY_LOC loc
             , dwh_owner.ORGANIZATIONS org          -- neni nutne, pro kontrolu
         where pt.pt_key = loc.pt_key
           and loc.nearest_org_key = org.org_key
           and loc.nearest_org_key is not null
           and loc.loc_type = 'K'
       ) src
    on (tgt.pt_key = src.pt_key)
  when matched then update
   set tgt.hbr_key_new = src.org_key,
       tgt.hbr_unified_id_new = src.org_unified_id,
       tgt.alg_step = '06'
 ;

 commit;
                           
exception
 when others then
    rollback;
    raise;
end;

/* HBR ALG STEP7 - Pobocka nejbliz trvale adrese */
procedure alg_step7
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is

begin

 set_cfg_params;

 merge --+ full(tgt)
  into etl_owner.STG_HBR_PARTY_ORG  tgt
 using (
        select pt.pt_key, loc.nearest_org_key as org_key, org.org_unified_id
          from (
                select pt_key
                  from etl_owner.STG_HBR_PARTY_ORG
                 where hbr_key_new is null
                   and add_months(P_EFFECTIVE_DATE, -1*C_HB_ALG_PERIOD) > nvl(PROTECTION_DATE, date '1000-01-01')
                   and party_type <> 'E'
               ) pt
             , etl_owner.STG_HBR_PARTY_LOC loc
             , dwh_owner.ORGANIZATIONS org          -- neni nutne, pro kontrolu
         where pt.pt_key = loc.pt_key
           and loc.nearest_org_key = org.org_key
           and loc.nearest_org_key is not null
           and loc.loc_type = 'T'
       ) src
    on (tgt.pt_key = src.pt_key)
  when matched then update
   set  hbr_key_new = src.org_key,
        hbr_unified_id_new = src.org_unified_id,
        alg_step = '07'
 ;

 commit;

exception
 when others then
    rollback;
    raise;
end;

/* HBR ALG STEP8 - Posledni zruseny produkt */
procedure alg_step8
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is

begin

 set_cfg_params;

 merge --+ full(tgt)
  into etl_owner.STG_HBR_PARTY_ORG  tgt
 using (
        select PT_KEY
             , ORG_KEY
             , ORG_UNIFIED_ID
          from (
                select pt.PT_KEY
                     , acc.ACC_KEY
                     , acc.ACCTP_KEY
                     , acc.ORG_KEY
                     , acc.ORG_UNIFIED_ID
                     , rel.REL_TO
                     , row_number() over (partition by pt.PT_KEY order by rel.REL_TO desc, acc.ACC_KEY) as RN
                  from (
                        select pt_key
                         from etl_owner.STG_HBR_PARTY_ORG
                        where hbr_key_new is null
                          and add_months(P_EFFECTIVE_DATE, -1*C_HB_ALG_PERIOD) > nvl(PROTECTION_DATE, date '1000-01-01')
                          and party_type <> 'E'           -- jeste nemaji pobocku
                       ) pt
                     , (
                        select pt_key
                             , ptori_key
                             , pt_unified_key
                             , ptori_unified_key
                          from dwh_owner.PARTIES
                         where ptori_key not in (4,99)         -- historicke instance ne
                           --and pt_deleted_flag = 'N'  --C1558332
                       ) inst
                     , (
                        select pt_key
                             , ptori_key
                             , acc_key
                             , acctp_key
                             , accpt_valid_from as rel_from
                             , accpt_valid_to as rel_to
                             , accpttp_unified_id as rel_typ
                          from dwh_owner.account_parties ap,
                               dwh_owner.account_party_types apt
                         where ap.accpttp_key = apt.accpttp_key
                           and apt.accpttp_unified_id in ('42','45')
                           and not(P_EFFECTIVE_DATE between ap.accpt_valid_from and ap.accpt_valid_to)
                           and P_EFFECTIVE_DATE > ap.accpt_valid_to
                       ) rel
                     , (
                        select x.acc_key
                             , x.acctp_key
                             , x.org_key
                             , org.org_unified_id
                          from (
                                select /*+ full(o)*/
                                       a.acc_key
                                     , a.acctp_key
                                     , LD_DWH_CRA_HBR_MAP.get_nearest (nvl(loc_geo_latitude,0), nvl(loc_geo_longitude,0), nvl(loc_geo_jtsk_x,0), nvl(loc_geo_jtsk_y,0)) as org_key
                                  from dwh_owner.all_accounts a
                                     , dwh_owner.organizations o
                                     , dwh_owner.locations l
                                 where a.org_key > 0
                                   and a.acctp_key not in (9999)   -- historicke ucty ne
                                   and a.org_key = o.org_key
                                   --and o.org_deleted_flag = 'Y'   --C1558332
                                   and o.loc_key = l.loc_key (+)
                                   and o.loc_source_system_id = l.loc_source_system_id (+)
                                   and l.loc_source_system_id (+) = 'AD0'
                               ) x
                             , dwh_owner.organizations org
                         where x.org_key = org.org_key
                           and x.org_key > 0
                       ) acc
                 where pt.pt_key = inst.pt_unified_key
                   and 4 = inst.ptori_unified_key
                   and inst.pt_key = rel.pt_key
                   and inst.ptori_key = rel.ptori_key
                   and rel.acc_key = acc.acc_key
                   and rel.acctp_key = acc.acctp_key
               )
         where RN = 1
       ) src
    on (tgt.PT_KEY = src.PT_KEY)
  when matched then update
   set hbr_key_new = src.org_key,
       hbr_unified_id_new = src.org_unified_id,
       alg_step = '08';

 commit;

exception
 when others then
    rollback;
    raise;
end;


/* HBR ALG STEP9 - Pobocka nejbliz adrese instancnich klientu (Step6 se tyka pouze unifikovanych klientu) */
procedure alg_step9
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is

begin

 set_cfg_params;

 merge --+ full(tgt)
  into etl_owner.STG_HBR_PARTY_ORG  tgt
 using (select y.pt_key,
               y.nearest_org_key,
               o.org_unified_id
          from (select x.pt_key, x.loc_geo_jtsk_x, x.loc_geo_jtsk_y, x.loc_geo_latitude, x.loc_geo_longitude,
                       etl_owner.LD_DWH_CRA_HBR_MAP.get_nearest (x.loc_geo_latitude, x.loc_geo_longitude, x.loc_geo_jtsk_x, x.loc_geo_jtsk_y) as nearest_org_key
                  from (select ptorg.pt_key,
                               loc.loc_geo_jtsk_x, loc.loc_geo_jtsk_y, loc.loc_geo_latitude, loc.loc_geo_longitude,
                               row_number() over (partition by ptorg.pt_key order by ps_inst.ptst_valid_from desc, (case pst.ptstp_unified_id when 'K' then 1 else 2 end) asc, ps_inst.ptst_key desc) as rnum
                          from etl_owner.STG_HBR_PARTY_ORG ptorg
                          join dwh_owner.parties p_inst
                            on p_inst.ptori_key not in (4, 64, 65, 66)
                           and ptorg.pt_key = p_inst.pt_unified_key
                           and p_inst.pt_deleted_flag = 'N'
                          join dwh_owner.PARTY_SITES ps_inst
                            on p_inst.ptori_key = ps_inst.ptori_key
                           and p_inst.pt_key = ps_inst.pt_key
                           and P_EFFECTIVE_DATE between ps_inst.ptst_valid_from and ps_inst.ptst_valid_to
                          join dwh_owner.PARTY_SITE_TYPES pst
                            on ps_inst.ptstp_key = pst.ptstp_key
                          join dwh_owner.LOCATIONS loc
                            on ps_inst.loc_key = loc.loc_key
                           and loc.loc_geo_jtsk_x > 0
                           and loc.loc_geo_jtsk_y > 0
                           and loc.loc_deleted_flag = 'N'
                         where ptorg.hbr_key_new is null
                           and add_months(P_EFFECTIVE_DATE, -1*C_HB_ALG_PERIOD) > nvl(ptorg.protection_date, date '1000-01-01')
                           and ptorg.party_type <> 'E'
                       ) x
                 where x.rnum = 1
               ) y
          join DWH_OWNER.organizations o
            on y.nearest_org_key = o.org_key
         where y.nearest_org_key > 0
       ) src
    on (tgt.PT_KEY = src.PT_KEY)
  when matched then update
   set hbr_key_new = src.nearest_org_key,
       hbr_unified_id_new = src.org_unified_id,
       alg_step = '09';

 commit;

exception
 when others then
    rollback;
    raise;
end;


/* HBR ALG STEP10 - Zachovani puvodni pobocky (v pripade, ze je validni) */
procedure alg_step10
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is

begin

 set_cfg_params;

 merge --+ full(tgt)
  into etl_owner.STG_HBR_PARTY_ORG  tgt
 using (
        select ptorg.PT_KEY
             , org.ORG_KEY
             , org.ORG_UNIFIED_ID
          from etl_owner.STG_HBR_PARTY_ORG ptorg
             , etl_owner.STG_HBR_ORG org
         where ptorg.HBR_KEY_NEW is null
           and add_months(P_EFFECTIVE_DATE, -1*C_HB_ALG_PERIOD) > nvl(PROTECTION_DATE, date '1000-01-01')
           and ptorg.PARTY_TYPE <> 'E'
           and ptorg.HBR_KEY_OLD = org.ORG_KEY
       ) src
    on (tgt.PT_KEY = src.PT_KEY)
  when matched then update
   set hbr_key_new = src.org_key,
       hbr_unified_id_new = src.org_unified_id,
       alg_step = '10';

 commit;

exception
 when others then
    rollback;
    raise;
end;


/*** PUBLIC PROCEDURES - APPLY NEW HOMEBRANCH ***/
/* Apply homebranch -- PARTIES  */
procedure main_pt
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is
  l_fullflg         char(1) := 'N';
begin

 select SUPPORT_FUNCTIONS.CHECK_HBR_FULL_DATE (P_EFFECTIVE_DATE) into l_fullflg from dual;

  merge /*+ use_hash (tgt src) */ into dwh_owner.parties partition (partition_4) tgt
  using
   (select pt_key,
           nvl(hbr_key_new,-1) as org_key
      from etl_owner.stg_hbr_party_org stg
     where nvl(hbr_unified_id_new,'HR0_69999999') <> hbr_unified_id_old
       and nvl(alg_step,decode(l_fullflg,'Y','XNA','XXX')) = nvl(alg_step,decode(l_fullflg,'Y','XNA','YYY')) --if increment, load records with filled alg_step only
    ) src
  on
   ( tgt.pt_key = src.pt_key )
  when matched then
    update set
      tgt.org_key = src.org_key,
      tgt.pt_updated_effective_date = p_effective_date,
      tgt.pt_updated_datetime = sysdate,
      tgt.pt_update_process_key = p_process_key
    where
      tgt.pt_organization_source_code != 'C' and
      tgt.org_key <> src.org_key;

 commit;

exception
 when others then
    rollback;
    raise;
end;

/* Apply homebranch - PARTY_ORGANIZATIONS  */
procedure main_ptorg
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is
  l_typ             integer;
  l_fullflg         char(1) := 'N';
begin

 select SUPPORT_FUNCTIONS.CHECK_HBR_FULL_DATE (P_EFFECTIVE_DATE) into l_fullflg from dual;

 -- get party org type = A
 select ptorgtp_key
   into l_typ
   from dwh_owner.party_organization_types
  where ptorgtp_source_id = 'A'
    and ptorgtp_source_system_id = 'EX0'
 ;

 merge into dwh_owner.PARTY_ORGANIZATIONS tgt
 using
 (select pt_key,
         nvl(hbr_key_new,-1) as org_key,
         l_typ as ptorgtp_key
    from etl_owner.stg_hbr_party_org stg
   where nvl(hbr_unified_id_new,'HR0_69999999') <> hbr_unified_id_old
     and nvl(alg_step,decode(l_fullflg,'Y','XNA','XXX')) = nvl(alg_step,decode(l_fullflg,'Y','XNA','YYY')) --if increment, load records with filled alg_step only
   minus
  select ptorg.pt_key,
         ptorg.org_key,
         ptorg.ptorgtp_key
    from dwh_owner.party_organizations ptorg
   where ptorg.ptorg_current_flag = 'Y'
     and ptori_key = 4
     and ptorg.ptorgtp_key = l_typ
  ) src
 on
  (tgt.pt_key = src.pt_key and
   tgt.ptorg_valid_from = p_effective_date and
   tgt.ptori_key = 4 and
   tgt.ptorgtp_key = l_typ)
 when matched then update set
      tgt.org_key = src.org_key,
      tgt.ptorg_source_system_id = 'OD0',
      tgt.ptorg_valid_to = to_date('01013000','ddmmyyyy'),
      tgt.ptorg_current_flag = 'Y',
      tgt.ptorg_updated_datetime = sysdate,
      tgt.ptorg_update_process_key = p_process_key
 when not matched then
   insert ( ptorg_key,
            pt_key,
            org_key,
            ptori_key,
            ptorgtp_key,
            ptorg_source_system_id,
            ptorg_valid_from,
            ptorg_valid_to,
            ptorg_current_flag,
            ptorg_inserted_datetime,
            ptorg_updated_datetime,
            ptorg_insert_process_key,
            ptorg_update_process_key)
    values
            (dwh_owner.party_organizations_s.nextval,
             src.pt_key,
             src.org_key,
             4,
             src.ptorgtp_key,
             'OD0',
             p_effective_date,
             to_date('01013000','ddmmyyyy'),
             'Y',
             sysdate,
             sysdate,
             p_process_key,
             p_process_key
            );

 -- close old
 merge into dwh_owner.party_organizations tgt
 using
 (select ptorg_key,
         ptorgtp_key,
         ptorg_valid_to,
         next_from
    from(select ptorg_key,
                ptori_key,
                pt_key,
                ptorg_valid_from,
                ptorg_valid_to,
                ptorgtp_key,
                lead(ptorg_valid_from) over (partition by pt_key, ptori_key, ptorgtp_key order by ptorg_valid_from) as next_from
           from dwh_owner.party_organizations
          where ptori_key = 4
            and ptorgtp_key = l_typ)
   where next_from is not null and next_from-1 < ptorg_valid_to
 ) src
 on
  (tgt.ptori_key = 4 and
   tgt.ptorg_key = src.ptorg_key and
   tgt.ptorgtp_key = src.ptorgtp_key)
 when matched then update set
   tgt.ptorg_valid_to = src.next_from - 1,
   tgt.ptorg_current_flag ='N',
   tgt.ptorg_updated_datetime = sysdate,
   tgt.ptorg_update_process_key = p_process_key;

 commit;

exception
 when others then
    rollback;
    raise;
end;

/* Apply homebranch - PARTY_CLASSF_USAGES  */
procedure main_ptclsus
 (p_process_key      in number default -1,
  p_effective_date   in date default to_date('10000101','YYYYMMDD'),
  p_load_partition   in varchar2 default null)
is
  l_alg00step   integer:= 0;
  l_fullflg     char(1) := 'N';
begin

select PTCLS_KEY into l_alg00step
  from dwh_owner.PARTY_CLASSIFICATIONS cls
 where 1=1
   and cls.PTCLS_SOURCE_ID = '00'
   and cls.PTCLSTP_KEY = 211
;

select SUPPORT_FUNCTIONS.CHECK_HBR_FULL_DATE(P_EFFECTIVE_DATE) into l_fullflg from dual;

  --MRG
 merge
  into dwh_owner.PARTY_CLASSF_USAGES trg
 using (
        (
        select PT_KEY
             , 4 as PTORI_KEY
             , cls.PTCLSTP_KEY
             , cls.PTCLS_KEY
             , 'OD0' as PTCLSUS_SOURCE_SYSTEM_ID
          from etl_owner.STG_HBR_PARTY_ORG stg
             , dwh_owner.PARTY_CLASSIFICATIONS cls
         where 1=1
           and ALG_STEP is not null
           and stg.ALG_STEP = cls.PTCLS_SOURCE_ID
           and cls.PTCLSTP_KEY = 211 --HBR_ALG_STEP
        UNION ALL
        select p.PT_KEY
             , p.PTORI_KEY
             , 211 as PTCLSTP_KEY --HBR_ALG_STEP
             , l_alg00step as PTCLS_KEY
             , 'OD0' as PTCLSUS_SOURCE_SYSTEM_ID
          from dwh_owner.PARTIES p
             , dwh_owner.PARTY_TYPES pttp
         where p.PTORI_KEY = 4
           and p.PT_ORGANIZATION_SOURCE_CODE = 'C'
           and p.PT_DELETED_FLAG = 'N'
           and p.PTTP_KEY = pttp.PTTP_KEY
           and pttp.PTTP_UNIFIED_ID <> 'E'   -- krome ESSO
        )
        MINUS
        select PT_KEY,
               PTORI_KEY,
               PTCLSTP_KEY,
               PTCLS_KEY,
               PTCLSUS_SOURCE_SYSTEM_ID
          from dwh_owner.PARTY_CLASSF_USAGES
         where PTORI_KEY = 4
           and PTCLSUS_CURRENT_FLAG = 'Y'
           and PTCLSTP_KEY = 211
       ) src
    ON (
        trg.PT_KEY = src.PT_KEY and
        trg.PTORI_KEY = src.PTORI_KEY and
        trg.PTCLSTP_KEY = src.PTCLSTP_KEY and
        trg.PTCLSTP_KEY = 211 and
        trg.PTCLSUS_VALID_FROM  = P_EFFECTIVE_DATE and
        trg.PTCLSUS_SOURCE_SYSTEM_ID = 'OD0'
       )
  WHEN MATCHED THEN UPDATE
   SET  trg.PTCLS_KEY = src.PTCLS_KEY,
        trg.PTCLSUS_UPDATED_DATETIME = sysdate,
        trg.PTCLSUS_UPDATE_PROCESS_KEY = P_PROCESS_KEY
  WHEN NOT MATCHED THEN INSERT (
        trg.PTCLSUS_KEY,
        trg.PT_KEY,
        trg.PTCLS_KEY,
        trg.PTORI_KEY,
        trg.PTCLSTP_KEY,
        trg.PTCLSUS_SOURCE_SYSTEM_ID,
        trg.PTCLSUS_VALID_FROM,
        trg.PTCLSUS_VALID_TO,
        trg.PTCLSUS_CURRENT_FLAG,
        trg.PTCLSUS_INSERTED_DATETIME,
        trg.PTCLSUS_UPDATED_DATETIME,
        trg.PTCLSUS_INSERT_PROCESS_KEY,
        trg.PTCLSUS_UPDATE_PROCESS_KEY
       )
VALUES (
        dwh_owner.PARTY_CLASSF_USAGES_S.nextval,
        src.PT_KEY,
        src.PTCLS_KEY,
        src.PTORI_KEY,
        src.PTCLSTP_KEY,
        src.PTCLSUS_SOURCE_SYSTEM_ID,
        P_EFFECTIVE_DATE,
        date '3000-01-01',
        'Y',
        sysdate,
        sysdate,
        P_PROCESS_KEY,
        P_PROCESS_KEY
       )
 ;

 if l_fullflg = 'Y' then
 -- FDL
 update dwh_owner.PARTY_CLASSF_USAGES tgt
    set PTCLSUS_VALID_TO = P_EFFECTIVE_DATE - 1
      , PTCLSUS_CURRENT_FLAG  = 'N'
      , PTCLSUS_UPDATED_DATETIME = sysdate
      , PTCLSUS_UPDATE_PROCESS_KEY = P_PROCESS_KEY
  where tgt.PTORI_KEY = 4
    and tgt.PTCLSUS_SOURCE_SYSTEM_ID = 'OD0'
    and tgt.PTCLSTP_KEY = 211
    and tgt.PTCLSUS_VALID_TO >= P_EFFECTIVE_DATE
    and PT_KEY not in
       (
        select PT_KEY
          from etl_owner.STG_HBR_PARTY_ORG stg
             , dwh_owner.PARTY_CLASSIFICATIONS cls
         where 1=1
           and ALG_STEP is not null
           and stg.ALG_STEP = cls.PTCLS_SOURCE_ID
           and cls.PTCLSTP_KEY = 211 --HBR_ALG_STEP
        UNION ALL
        select p.PT_KEY
          from dwh_owner.PARTIES p
             , dwh_owner.PARTY_TYPES pttp
         where p.PTORI_KEY = 4
           and p.PT_ORGANIZATION_SOURCE_CODE = 'C'
           and p.PT_DELETED_FLAG = 'N'
           and p.PTTP_KEY = pttp.PTTP_KEY
           and pttp.PTTP_UNIFIED_ID <> 'E'
       );

 end if;

 commit;

exception
 when others then
    rollback;
    raise;
end;


END;
/

In [0]:
/* Oracle's CONNECT BY hierarchical queries need to be converted to recursive CTEs in Databricks*/
select distinct org_key
                  from gap_catalog.dwh_owner.organizations
                 where org_deleted_flag = 'Y'
               connect by prior org_key = org_master_key
                 start with org_master_key in
                       (select org_key
                          from gap_catalog.dwh_owner.organizations oh
                         where org_id2 in
                             (select usepar_value_1
                               from gap_catalog.dwh_owner.user_config_parameters
                              where usepar_group_id = 'D4ORG'
                                and usepar_id like 'D4_ORG_ID2_%')
                       )

In [0]:
WITH RECURSIVE hierarchy AS (
  -- Base case: Start with organizations matching D4ORG parameters
  SELECT 
    org_key, 
    org_master_key, 
    org_deleted_flag,
    1 as level
  FROM gap_catalog.dwh_owner.organizations
  WHERE org_master_key IN (
    SELECT org_key
    FROM gap_catalog.dwh_owner.organizations oh
    WHERE org_id2 IN (
      SELECT usepar_value_1
      FROM gap_catalog.dwh_owner.user_config_parameters
      WHERE usepar_group_id = 'D4ORG'
        AND usepar_id LIKE 'D4_ORG_ID2_%'
    )
  )
   UNION ALL
   -- Recursive case: Find children of current level
  SELECT 
    o.org_key, 
    o.org_master_key, 
    o.org_deleted_flag,
    h.level + 1
  FROM gap_catalog.dwh_owner.organizations o
  JOIN hierarchy h ON o.org_master_key = h.org_key
  WHERE h.level < 10  -- Prevent infinite recursion
)
SELECT DISTINCT org_key
FROM hierarchy
WHERE org_deleted_flag = 'Y';